# Using the Edge Impulse Python SDK with TensorFlow and Keras

<!--- Do not modify the markdown for this example directly! It is generated from a notebook in https://github.com/edgeimpulse/notebooks --->

[![View in Edge Impulse docs](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/ei-badge.svg)](https://docs.edgeimpulse.com/docs/tutorials/ml-and-data-engineering/ei-python-sdk/python-sdk-with-tf-keras)
[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-with-tf-keras.ipynb)
[![View on GitHub](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/badge-view-on-github.svg)](https://github.com/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-with-tf-keras.ipynb)
[![Download notebook](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/badge-download-notebook.svg)](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/notebooks/python-sdk-with-tf-keras.ipynb)

This notebook demonstrates how to train a model using TensorFlow and deploy it using the Edge Impulse Python SDK. Here, we customize it to work with the Edge Impulse `openmvh7p` device and a specific API key.

In [ ]:
# Install necessary packages if not installed already
!python -m pip install tensorflow==2.12.0 edgeimpulse

In [ ]:
from tensorflow import keras
import edgeimpulse as ei

## Configure API Key and Device
Set your Edge Impulse API key and specify your target device (`openmvh7p`).

In [ ]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842"
device_name = "openmvh7p"
labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
num_classes = len(labels)
deploy_filename = "my_model_cpp.zip"

## Load and Prepare Data
We will use the MNIST dataset for training.

In [ ]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = keras.utils.normalize(x_train, axis=1)
x_test = keras.utils.normalize(x_test, axis=1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
input_shape = x_train[0].shape

In [ ]:
# Build the model
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu', input_shape=input_shape),
    keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train,
          y_train,
          epochs=5)

In [ ]:
# Evaluate model on test set
score = model.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {score[1]}")

## Profile and Deploy the Model for the `openmvh7p` Device
Now we profile the model and prepare it for deployment.

In [ ]:
# Profile the model for your specific device
devices = ei.model.list_profile_devices()
device_id = next((d['device'] for d in devices if d['device'] == device_name), None)
if device_id:
    profile = ei.model.profile_model(model, device=device_id)
    print(profile)
else:
    print(f"Device '{device_name}' not found in available devices.")

In [ ]:
# Convert model to Edge Impulse compatible format
ei.model.convert_to_cpp(model, deploy_filename, device=device_name)
print(f"Model converted and saved as {deploy_filename}")